In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from functools import reduce

In [2]:
import os
os.getcwd()

'/Users/fangduan/Downloads'

In [3]:
M1 = pd.read_csv('lightGBM_test_predictions_rank_mean.csv')
M2 = pd.read_csv('logistic_0.750.csv')
M3 = pd.read_csv('pure_gp_0.787.csv')
M4 = pd.read_csv('ffnn_rank_gauss_0.775.csv')
M5 = pd.read_csv('tidy_xgb_0.78847.csv')

In [4]:
M2 = M2.drop(columns = 'Unnamed: 0')

In [5]:
def calculate_rank(predictions):
    rank = (1 + predictions.rank().values) / (predictions.shape[0] + 1)
    return rank
M3['TARGET'] = calculate_rank(M3['TARGET'])
M4['TARGET'] = calculate_rank(M4['TARGET'])
M5['TARGET'] = calculate_rank(M5['TARGET'])

In [6]:
def merge_dataframes(dfs, merge_keys):
    dfs_merged = reduce(lambda left,right: pd.merge(left, right, on=merge_keys), dfs)
    return dfs_merged

In [7]:
dfs = [M1,M2,M3,M4,M5]
merge_keys=['SK_ID_CURR']
df = merge_dataframes(dfs, merge_keys=merge_keys)

In [8]:
df.columns = ['SK_ID_CURR','T1','T2','T3','T4','T5']
df.head()

,SK_ID_CURR,T1,T2,T3,T4,T5
0,100001,0.466456,0.612333,0.684480,0.472048,0.354252
1,100005,0.842515,0.312973,0.904421,0.894164,0.796205
2,100013,0.420531,0.717888,0.357493,0.321941,0.238055
3,100028,0.508073,0.760559,0.278408,0.617438,0.473074
4,100038,0.869041,0.296655,0.915663,0.824762,0.827018


In [9]:
df.corr()

,SK_ID_CURR,T1,T2,T3,T4,T5
SK_ID_CURR,1.000000,-0.003980,0.006842,-0.005781,-0.011333,-0.004777
T1,-0.003980,1.000000,-0.835998,0.917175,0.894350,0.937766
T2,0.006842,-0.835998,1.000000,-0.858902,-0.851798,-0.819606
T3,-0.005781,0.917175,-0.858902,1.000000,0.911605,0.902385
T4,-0.011333,0.894350,-0.851798,0.911605,1.000000,0.883615
T5,-0.004777,0.937766,-0.819606,0.902385,0.883615,1.000000


In [10]:
B_prob = 0.5 * df['T1'] + 0.1 * df['T2'] + 0.1 * df['T3'] + 0 * df['T4'] + 0.3 * df['T5']
B_prob.head(10)

0    0.476400
1    0.841003
2    0.353181
3    0.451640
4    0.865759
5    0.531523
6    0.029388
7    0.262776
8    0.146425
9    0.721020
dtype: float64

In [11]:
SUB = pd.DataFrame()
SUB['SK_ID_CURR'] = df['SK_ID_CURR']
SUB['TARGET'] = B_prob
SUB.to_csv('Blender_rank.csv', index=False)

In [12]:
## read saved 1174 features
# import pandas as pd
# from sklearn import externals as ext
# temp = ext.joblib.load('../Documents/Projects/solution 5/feature_joiner_train')
# train = pd.DataFrame(temp['features'])

In [13]:
# train.to_csv('train.csv.gz', compression='gzip', sep = ',', index = False) # train.to_csv('train.csv') does not work

In [14]:
# temp = ext.joblib.load('../Documents/Projects/solution 5/feature_joiner_test')
# test = pd.DataFrame(temp['features'])

In [15]:
# test.to_csv('test.csv', index = False)